In [1]:
import sys; sys.path.append("../")

In [2]:
import pandas as pd
from utils_analysis import make_run_overview_csv

make_run_overview_csv()
runs_df = pd.read_csv("run_overview.csv", index_col=0)
runs_df

Saving overview in /home/cbarkhof/code-thesis/NewsVAE/final-analysis/run_overview.csv


,run_name,clean_name,dataset,optimisation,drop,target_rate,matrix,emb,mem
0,2021-05-24-PTB | CYC-FB-0.5 | matrix-run-15:44:43,PTB | CYC-FB-0.5 | matrix,PTB,CYC-FB-0.5,0.0,0.5,True,False,False
1,2021-05-24-PTB | CYC-FB-0.5 | matrix-memory-ru...,PTB | CYC-FB-0.5 | matrix-mem,PTB,CYC-FB-0.5,0.0,0.5,True,False,True
2,2021-05-21-PTB | VAE | memory-run-05:50:37,PTB | VAE | mem,PTB,VAE,0.0,0.0,False,False,True
3,2021-05-21-PTB | VAE | memory | DROP 40-run-05...,PTB | VAE | mem | DROP 40,PTB,VAE,0.4,0.0,False,False,True
4,2021-05-21-PTB | VAE | memory-embeddings-run-0...,PTB | VAE | mem-emb,PTB,VAE,0.0,0.0,False,True,True
5,2021-05-21-PTB | VAE | memory-embeddings | DRO...,PTB | VAE | mem-emb | DROP 40,PTB,VAE,0.4,0.0,False,True,True
6,2021-05-22-PTB | CYC-FB-0.5 | memory-run-02:03:27,PTB | CYC-FB-0.5 | mem,PTB,CYC-FB-0.5,0.0,0.5,False,False,True
7,2021-05-22-PTB | CYC-FB-0.5 | memory | DROP 40...,PTB | CYC-FB-0.5 | mem | DROP 40,PTB,CYC-FB-0.5,0.4,0.5,False,False,True
8,2021-05-22-PTB | CYC-FB-0.5 | memory-embedding...,PTB | CYC-FB-0.5 | mem-emb,PTB,CYC-FB-0.5,0.0,0.5,False,True,True
9,2021-05-22-PTB | CYC-FB-0.5 | memory-embedding...,PTB | CYC-FB-0.5 | mem-emb | DROP 40,PTB,CYC-FB-0.5,0.4,0.5,False,True,True


In [4]:
from utils_train import load_from_checkpoint
from utils_analysis import get_best_checkpoint, save_last_pareto_dict_to_pickle

save_last_pareto_dict_to_pickle()

for row_idx, row in runs_df.iterrows():
    
    path, best_epoch = get_best_checkpoint(row["run_name"])
    if path is not None:
        loss_term_manager = load_from_checkpoint(path, world_master=True, ddp=False, device_name="cuda:0", 
                                                 evaluation=True, return_loss_term_manager=False)
    
    
        break

2021-05-24-PTB | CYC-FB-0.5 | matrix-run-15:44:43
error: list index out of range
2021-05-24-PTB | VAE | matrix-memory-run-15:32:00
error: list index out of range
no weighted pareto scores for this run yet 2021-05-24-PTB | CYC-FB-0.5 | matrix-run-15:44:43


UnboundLocalError: local variable 'best_epoch' referenced before assignment